In [1]:
# Install a pip package in the current Jupyter kernel - Uncomment if needed
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install bs4
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install selenium
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install urllib
# !{sys.executable} -m pip install re
# !{sys.executable} -m pip install nbimporter

In [14]:
# Import the relevant packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import numpy as np
from urllib.parse import urljoin
from urllib.error import HTTPError, URLError
from urllib.request import urlopen, Request
import re
import nbimporter
import json
# import functions from different scraping_util notebook; Make sure both scripts are in the same folder
%run scraping_util.ipynb

In [3]:
# import data and filter for relevant BL
data = pd.read_csv(r'C:\Users\marius\Documents\pol_viol\pol_viol\liste_gemeinden_ger.csv', delimiter=';')
data = data.dropna(thresh=8).reset_index(drop=True)
data['AGS'] = '0' + data['Land'].astype(str) + data['RB'].astype(int).astype(str) + data['Kreis'].astype(int).astype(str) + '0'+ data['Gem'].astype(int).astype(str)
data.loc[data['Gem'] < 10, 'AGS'] = '0' + data['Land'].astype(str) + data['RB'].astype(int).astype(str) + data['Kreis'].astype(int).astype(str) + '00' + data['Gem'].astype(int).astype(str)
data_ni = data[data['Land'] == 3]
data_scrape = data_ni[['Gemeindename', 'plz', 'AGS']]
data_scrape.head()

,Gemeindename,plz,AGS
1109,Braunschweig,38100.0,0311000
1110,Salzgitter,38226.0,0312000
1111,Wolfsburg,38440.0,0313000
1112,Gifhorn,38518.0,03151009
1113,Sassenburg,38524.0,03151025


In [6]:
# set up webdriver and search field element
# regularly check for the right version of webdriver
# chrome version: 99.0.4844.84
# chrome driver: 99.0.4844.51
# Important to keep in mind: both need to be of version 97.
d=webdriver.Chrome(r"C:\Users\marius\Documents\pol_viol\pol_viol\chromedriver.exe")
d.get('https://wahlen.votemanager.de/#')
e = d.find_element_by_id('suchfeld')

# search for each municipality and get the url to the municipality specific overview page
data_scrape['url'] = data_scrape['Gemeindename'].apply(get_url1)
data_scrape.reset_index(drop=True, inplace=True)
data_scrape = data_scrape[data_scrape.astype(str)['url']!='[]'].reset_index(drop=True)

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [7]:
data_scrape['results'] = data_scrape['url'].str.len()
# Find the relevant link and ignore mismatches
for row in data_scrape.iterrows():
    if row[1][4] > 1:
        for link in row[1][3]:
            if row[1][2] in link:
                data_scrape.loc[(data_scrape['Gemeindename'] == row[1][0]), 'url'] = link

In [8]:
# Create links for the three last election cycles
data_scrape['url_copy'] = data_scrape['url'].astype(str)
data_exp = data_scrape['url_copy'].str.replace("\['", "").str.replace("'\]", "").str.split('/',expand=True)
data_scrape['url_21'] = data_exp[0]+ '//' + data_exp[2]+ '/' + '20210912/' + data_exp[3] + '/html5/' + data_exp[4]
data_scrape['url_16'] = data_exp[0]+ '//' + data_exp[2]+ '/' + '20160911/' + data_exp[3] + '/html5/' + data_exp[4]

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
def get_url3(url2):
    """ Function to get three possible links for every election. Each of 'matches' defines a possible link 
        specification. Only input is the link column.
    """
    
    r = requests.get(url2) 
    soup = BeautifulSoup(r.text)
    base = url2
    hrefs = [link.get('href') for link in soup.find_all('a')]
    
    matches = ['Stadtverordnetenwahl - ', 'Gemeindewahl - ', 'Gemeinderatswahl-',
              'Stadtratswahl -']
    
    href_2=[href for href in hrefs if any(x in href for x in matches)]
    
    if href_2:
        s = href_2.pop()
        return urljoin(base,s)
    else:
        return np.nan

In [147]:
data_scrape

,Gemeindename,plz,AGS,url,results,url_copy,url_21,url_16,AGS_len
0,Salzgitter,38226.0,0312000,[https://www.salzgitter.de/wahlen/ergebnisse/0...,1,['https://www.salzgitter.de/wahlen/ergebnisse/...,https://www.salzgitter.de/20210912/wahlen/html...,https://www.salzgitter.de/20160911/wahlen/html...,7
1,Wolfsburg,38440.0,0313000,[https://wahlen.wolfsburg.de/#],1,['https://wahlen.wolfsburg.de/#'],NaN,NaN,7
2,Gifhorn,38518.0,03151009,https://votemanager.kdo.de/03151009/index.html,2,https://votemanager.kdo.de/03151009/index.html,https://votemanager.kdo.de/20210912/03151009/h...,https://votemanager.kdo.de/20160911/03151009/h...,8
3,Sassenburg,38524.0,03151025,[https://votemanager.kdo.de/03151025/index.html],1,['https://votemanager.kdo.de/03151025/index.ht...,https://votemanager.kdo.de/20210912/03151025/h...,https://votemanager.kdo.de/20160911/03151025/h...,8
4,Wittingen,29378.0,03151040,[https://votemanager.kdo.de/03151040/index.html],1,['https://votemanager.kdo.de/03151040/index.ht...,https://votemanager.kdo.de/20210912/03151040/h...,https://votemanager.kdo.de/20160911/03151040/h...,8
...,...,...,...,...,...,...,...,...,...
961,Neuschoo,26487.0,03462011,[https://votemanager.kdo.de/034625402/index.html],1,['https://votemanager.kdo.de/034625402/index.h...,https://votemanager.kdo.de/20210912/034625402/...,https://votemanager.kdo.de/20160911/034625402/...,8
962,Ochtersum,26489.0,03462012,[https://votemanager.kdo.de/034625402/index.html],1,['https://votemanager.kdo.de/034625402/index.h...,https://votemanager.kdo.de/20210912/034625402/...,https://votemanager.kdo.de/20160911/034625402/...,8
963,Schweindorf,26556.0,03462013,[https://votemanager.kdo.de/034625402/index.html],1,['https://votemanager.kdo.de/034625402/index.h...,https://votemanager.kdo.de/20210912/034625402/...,https://votemanager.kdo.de/20160911/034625402/...,8
964,Utarp,26556.0,03462016,[https://votemanager.kdo.de/034625402/index.html],1,['https://votemanager.kdo.de/034625402/index.h...,https://votemanager.kdo.de/20210912/034625402/...,https://votemanager.kdo.de/20160911/034625402/...,8


In [65]:
data_scrape['AGS_len'] = 8
for row in data_scrape.iterrows():
    if len(row[1][2]) < 8:
        data_scrape.loc[data_scrape.index == row[0], 'AGS_len'] = 7
        
link_list_low_level = []

,Gemeindename,plz,AGS,url,results,url_copy,url_21,url_16,AGS_len
0,Salzgitter,38226.0,0312000,[https://www.salzgitter.de/wahlen/ergebnisse/0...,1,['https://www.salzgitter.de/wahlen/ergebnisse/...,https://www.salzgitter.de/20210912/wahlen/html...,https://www.salzgitter.de/20160911/wahlen/html...,7
1,Wolfsburg,38440.0,0313000,[https://wahlen.wolfsburg.de/#],1,['https://wahlen.wolfsburg.de/#'],NaN,NaN,7
717,Delmenhorst,27749.0,0341000,[https://votemanager.kdo.de/03401000/index.html],1,['https://votemanager.kdo.de/03401000/index.ht...,https://votemanager.kdo.de/20210912/03401000/h...,https://votemanager.kdo.de/20160911/03401000/h...,7
718,Emden,26721.0,0342000,[https://votemanager.kdo.de/03402000/index.html],1,['https://votemanager.kdo.de/03402000/index.ht...,https://votemanager.kdo.de/20210912/03402000/h...,https://votemanager.kdo.de/20160911/03402000/h...,7
719,Oldenburg (Oldenburg),26105.0,0343000,[https://votemanager.kdo.de/03453003/index.htm...,6,['https://votemanager.kdo.de/03453003/index.ht...,https://votemanager.kdo.de/20210912/03453003/h...,https://votemanager.kdo.de/20160911/03453003/h...,7
720,Osnabrück,49074.0,0344000,[https://votemanager.kdo.de/03459000/index.htm...,2,['https://votemanager.kdo.de/03459000/index.ht...,https://votemanager.kdo.de/20210912/03459000/h...,https://votemanager.kdo.de/20160911/03459000/h...,7
721,Wilhelmshaven,26382.0,0345000,[https://www.wilhelmshaven.de/wahlen/03405000/...,1,['https://www.wilhelmshaven.de/wahlen/03405000...,https://www.wilhelmshaven.de/20210912/wahlen/h...,https://www.wilhelmshaven.de/20160911/wahlen/h...,7


In [143]:
# 2021 Gemeindelevel
link_list_low_level = []
for row in data_scrape[2:3].iterrows():
    ags = row[1][2]
    print(ags)
    for lid in range(2001, 2999):
        link = "https://votemanager.kdo.de/20210912/"+ str(ags) +"/praesentation/ergebnis.html?wahl_id=225&stimmentyp=0&id=ebene_8_id_" + str(lid)
        
        response = requests.get(link)

        try:
            soup = BeautifulSoup(response.text)
            link_list_low_level.append(soup)
        except:
            TypeError
            
    

03151009


In [82]:
len(data_manual_ni["url_21_clean"].isna())

967

In [7]:
data_manual_ni = pd.read_excel(r"C:\Users\Marius\OneDrive - Istituto Universitario Europeo\Documents\pol_viol\pol_viol\list_niedersachsen_new.xlsx")
data_manual_ni["url_21_clean"][1] + "#collapse2"

'https://www.salzgitter.de/wahlen/ergebnisse/Wahl-2021-09-12/03102000/praesentation/ergebnis.html?wahl_id=5&stimmentyp=0&id=ebene_3_id_1'

In [87]:
d=webdriver.Chrome(r"C:\Users\Marius\OneDrive - Istituto Universitario Europeo\Documents\pol_viol\pol_viol\chromedriver.exe")
d.get(data_manual_ni["url_21_clean"][1])

<ipython-input-87-f44046579aba>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d=webdriver.Chrome(r"C:\Users\Marius\OneDrive - Istituto Universitario Europeo\Documents\pol_viol\pol_viol\chromedriver.exe")


In [105]:
#elem = d.find_elements(By.XPATH, "//tbody/tr/td")#/tr[3]/td[2]/div/a[@class='link-info collapsed']")
elem[70].click()

In [94]:
rt = requests.get(elem[1]) 
BeautifulSoup(rt.text)

MissingSchema: Invalid URL '<selenium.webdriver.remote.webelement.WebElement (session="d10e28c3b10105a8f6aaad8bd524883e", element="59cd6f49-788a-4c6e-8498-8171ce5b8a32")>': No schema supplied. Perhaps you meant http://<selenium.webdriver.remote.webelement.WebElement (session="d10e28c3b10105a8f6aaad8bd524883e", element="59cd6f49-788a-4c6e-8498-8171ce5b8a32")>?

In [45]:
d.find_elements(By.XPATH, "//div/a[@class='link-info collapsed']")
d

<selenium.webdriver.chrome.webdriver.WebDriver (session="48149560eda63a5c3d135eac56f9d4ee")>